In [ ]:
# imports
import requests
from bs4 import BeautifulSoup
import pandas as pd

url = "https://www.ccny.cuny.edu/registrar/fall"
default_year = ',  2021' # necessary to fill in blank years

In [ ]:
html_doc = requests.get(url).text
soup = BeautifulSoup(html_doc)
# print(soup.prettify())
table = soup.find_all('table')[0]
# print(table)

rows = table.find_all('tr')
# print(rows)

data = {
    "date": [],
    "dow": [], # day of week
    "text": []
}

def clean_date(s: str) -> str:
    start = s.strip().split('-')[0]
    if ',' not in start:
        start += default_year
    return pd.to_datetime(start.strip(), format="%M %d, %Y").date()